In [1]:
%env CUDA_LAUNCH_BLOCKING=1
%env CUBLAS_WORKSPACE_CONFIG=:4096:8
%load_ext line_profiler
%load_ext autoreload
%autoreload 2
from GraPL import GraPL_Segmentor, side_by_side, view_multichannel, PatchDL, GraPLNet, segment_voc
import glob
import tqdm
import matplotlib.pyplot as plt
from GraPL.evaluate import *
import numpy as np
import warnings
import torch
import pandas as pd
import os
warnings.filterwarnings("ignore")

env: CUDA_LAUNCH_BLOCKING=1
env: CUBLAS_WORKSPACE_CONFIG=:4096:8


In [13]:
with open("datasets/PascalVOC2012/VOC2012/ImageSets/Segmentation/val.txt", "r") as f:
    val_image_ids = f.read().split("\n")

segmentation_mode = "SegmentationObject"

params = {
    'iterations': 4,
    'k': 14,
    'd': 32,
    'lambda_': 64,
    'subset_size': 0.5,
    'max_epochs': 40,
    'min_epochs': 12,
    'n_filters': 32,
    'bottleneck_dim': 8,
    'compactness': 0.1,
    'sigma': 10,
    'seed': 0,
    'use_continuity_loss': True,
    'continuity_range': 1,
    'continuity_p': 1,
    'continuity_weight': 2,
    'use_min_loss': True,
    'use_coords': False,
    'use_embeddings': False,
    'use_color_distance_weights': True,
    'initialization_method': 'slic',
    'use_fully_connected': True,
    'use_collapse_penalty': False,
    'use_cold_start': False,
    'num_layers': 3,
    'use_graph_cut': True
}

In [14]:
def run_experiment(base_params, num_trials=2):
    results_dir = f'experiment_results/naive_voc'
    if not os.path.exists(results_dir):
        os.makedirs(results_dir)
    aggregate_scores = {}
    trials = []
    mean_over_trials = {}
    with tqdm.tqdm(total=num_trials * len(val_image_ids)) as progress_bar:
        for trial_num in range(num_trials):
            params = base_params.copy()
            params['seed'] = trial_num
            trial_results_dir = f'{results_dir}/{trial_num}'
            trial_scores = segment_voc(results_dir=trial_results_dir, debug_num=-1, resume=True, progress_bar=progress_bar, **params)
            image_ids = list(trial_scores.keys())
            metrics = trial_scores[image_ids[0]].keys()
            trial_scores = {metric: np.mean([trial_scores[id][metric] for id in image_ids]) for metric in metrics}
            trials.append(trial_scores)
    for metric in trials[0]:
        mean_over_trials[metric] = np.mean([trial[metric] for trial in trials])
    aggregate_scores = mean_over_trials
    return aggregate_scores

run_experiment(params, num_trials=10)

  1%|          | 159/14500 [06:49<10:15:57,  2.58s/it]


KeyboardInterrupt: 